# Asymmetry Detection Models
This notebook shows the process of load and train of a LSTM neuronal network to classify the asymmetry laterality of a PD patient or a control. In this notebook
we will show different implementations of LSTMs (Normal and bidirectional) and with different hyperparameters.

## Requiered Imports

In [1]:
import pickle
import datetime

import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard


from src.model.asa_models import build_basic_lstm, build_2layer_lstm

from src.settings import ROOT_DIR
from src.utils.data_split import get_features_target

C:\Users\julia\AppData\Roaming\Python\Python310\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


KeyboardInterrupt: 

## Data load

In [2]:
train_data_path = ROOT_DIR / 'data' / 'processed' / 'asa' / 'train.pkl'
val_data_path = ROOT_DIR / 'data' / 'processed' / 'asa' / 'val.pkl'

with open(train_data_path, 'rb') as file:
    train_data = pickle.load(file)

with open(val_data_path, 'rb') as file:
    val_data = pickle.load(file)

In [3]:
len(train_data)

124

In [4]:
X_train, y_train = get_features_target(train_data)
X_val, y_val = get_features_target(val_data)

## Tensor Board
This tool will help us to supervise the experimentation process of all the experimental models described in this notebook

In [5]:
%load_ext tensorboard
%tensorboard --logdir ../results/asa/lightning_logs

Reusing TensorBoard on port 6006 (pid 17168), started 23:54:10 ago. (Use '!kill 17168' to kill it.)

In [0]:
log_dir  =  str(ROOT_DIR) + '/results/asa/lightning_logs' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir , histogram_freq=1)

## Models

In [0]:
checkpoint_filepath = str(ROOT_DIR) + '/results/asa/tmp/ckpt/checkpoint.model.keras'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

### Simple LSTM
This model is a 64-lstm with any hidden layers and just one dense layer to infer the output.
Optimizer: *Adam*
Loss: *binary_crossentropy*

In [6]:
BATCH_SIZE = 64
EPOCHS = 24

In [9]:
model = build_basic_lstm()
model.name

'sequential_2'

In [8]:
# Train the model
history_model = model.fit(
    X_train, y_train, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE,
    callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/24
2/2 [==============================] - 318s 68s/step - loss: 0.6941 - accuracy: 0.6613
Epoch 2/24
2/2 [==============================] - 216s 73s/step - loss: 0.6895 - accuracy: 0.6613
Epoch 3/24
2/2 [==============================] - 213s 76s/step - loss: 0.6858 - accuracy: 0.6613
Epoch 4/24
2/2 [==============================] - 226s 91s/step - loss: 0.6795 - accuracy: 0.6613
Epoch 5/24
2/2 [==============================] - 175s 49s/step - loss: 0.6722 - accuracy: 0.6613
Epoch 6/24
2/2 [==============================] - 210s 83s/step - loss: 0.6657 - accuracy: 0.6613
Epoch 7/24
2/2 [==============================] - 221s 91s/step - loss: 0.6496 - accuracy: 0.6613
Epoch 8/24
2/2 [==============================] - 216s 89s/step - loss: 0.6393 - accuracy: 0.6613
Epoch 9/24
2/2 [==============================] - 225s 90s/step - loss: 0.6495 - accuracy: 0.6613
Epoch 10/24
2/2 [==============================] - 219s 95s/step - loss: 0.6355 - accuracy: 0.6694
Epoch 11/24
2/2 [==

### 2 Hidden layers LSTM

In [0]:
lstm = build_2layer_lstm()

In [0]:
lstm.summary()

In [0]:
np.asarray(X_train, dtype=object).shape

In [0]:
lstm.fit(x=np.asarray(X_train, dtype=object), 
         y=np.asarray(y_train).astype(np.float32), 
         batch_size=BATCH_SIZE, 
         epochs=EPOCHS,
         validation_data=(X_val, y_val),
         callbacks=[tensorboard_callback, model_checkpoint_callback])